In [6]:
from acquire import scrape_github_data
import pandas as pd
import numpy as np
import nltk
import GB_prepare

In [2]:
REPOS = scrape_github_data()

In [3]:
repos = pd.DataFrame(REPOS)

In [12]:
len(repos[repos.original_length==0])

38

In [7]:
GB_prepare.prepare_df(repos ,'readme_contents' , extra_words = ['also', '&#9', 'e', 'f', 'ou', 'et', 'n', '1', "'", ';', '3', 'e', 'p'])

,repo,language,readme_contents,clean,stemmed,lemmatized,original_length,stem_length,lem_length,original_word_count,stemmed_word_count,lemmatized_word_count
0,zmb3/spotify,Go,\nSpotify\n=======\n\n[![GoDoc](https://godoc....,spotify godochttpsgodocorggithubcomzmb3spotify...,spotifi godochttpsgodocorggithubcomzmb3spotify...,spotify godochttpsgodocorggithubcomzmb3spotify...,3918,2352,2640,503,276,276
1,spotDL/spotify-downloader,Python,<!--- mdformat-toc start --slug=github --->\n\...,mdformattoc start sluggithub editing readme en...,mdformattoc start sluggithub edit readm ensur ...,mdformattoc start sluggithub editing readme en...,4300,2834,3021,471,277,277
2,Spotifyd/spotifyd,Rust,# Spotifyd <!-- omit in toc -->\n<!-- ALL-CONT...,spotifyd omit toc allcontributorsbadgestart re...,spotifyd omit toc allcontributorsbadgestart re...,spotifyd omit toc allcontributorsbadgestart re...,2338,1612,1707,235,140,140
3,Rigellute/spotify-tui,Rust,# Spotify TUI\n\n![Continuous Integration](htt...,spotify tui continuous integrationhttpsgithubc...,spotifi tui continu integrationhttpsgithubcomr...,spotify tui continuous integrationhttpsgithubc...,59878,32960,34049,3857,2495,2495
4,JohnnyCrazy/SpotifyAPI-NET,C#,"\n<h1 align=""center"">\n <p align=""center"">Spo...",h1 aligncenter aligncenterspotifyapinetp hrefh...,h1 aligncent aligncenterspotifyapinetp hrefhtt...,h1 aligncenter aligncenterspotifyapinetp hrefh...,2782,1881,2009,272,169,169
...,...,...,...,...,...,...,...,...,...,...,...,...
995,root-host/Spotify-AdBlock,None,,,,,0,0,0,0,0,0
996,anhjesslyn/spotify-account-creator,PHP,# Spotify Account Creator\n![license](https://...,spotify account creator licensehttpsimgshields...,spotifi account creator licensehttpsimgshields...,spotify account creator licensehttpsimgshields...,809,582,602,73,47,47
997,mattiasahlsen/spotify-queue,JavaScript,# spotify-app\n\n## deployed on\nhttps://colla...,spotifyapp deployed httpscollabqueuecom projec...,spotifyapp deploy httpscollabqueuecom project ...,spotifyapp deployed httpscollabqueuecom projec...,355,216,247,48,27,27
998,valeriangalliat/spotify-to-youtube,JavaScript,# Spotify to YouTube [![npm version](http://im...,spotify youtube npm versionhttpimgshieldsionpm...,spotifi youtub npm versionhttpimgshieldsionpmv...,spotify youtube npm versionhttpimgshieldsionpm...,2359,1568,1639,301,178,178


In [ ]:
nltk.download('wordnet')

In [ ]:
import dans_prepare

In [ ]:
dans_prepare.nlp_prep2(repos)